# C33/C34 Beatmungsgeräte
> Manuel Vazquez  
> Christian Spiller  
> Michael Wild

## Imports

In [1]:
import pandas as pd

## Load Data

## Cleanup and Pre-Processing of Data

## Functions

In [66]:
def calculate(inc, alpha, tau_e, tau_a):
    '''Model for the estimation of ventilator occupation given incidence time series
    
    Parameters
    ----------
    inc: pd.DataFrame
      DataFrame with incidence numbers with a multi-level index. Levels must be:
          * level=0: int, week numbers, named 't'
          * level=1: str, age-group, named 'age_group'
          * level=2: str, vaccination status, named 'vacc_status'
    alpha: pd.DataFrame
      Model parameters representing the _conversion factors_ from the incidence
      numbers to the number of patients requiring ventilation. The multi-level
      index must contain a value in the range [0, 1] for each combination of
      age-group and vaccination status.
    tau_e: int
      Model parameter describing the number of weeks that passes between
      infection and a patient being attached to a ventilator.
    tau_a: int
      Model parameter describing the number of weeks that passes between
      infection and a patient either no longer requires the ventilator
      or is deceased.
    
    Returns
    -------
    pd.DataFrame
      A time-series predicting the ventilator occupancy.
    '''
    
    # build grouping by age and vaccination status
    inc_grp = inc.groupby(level=['age_group', 'vacc_status'])
    # shift the groups and calculate the difference
    delta = inc_grp.shift(tau_e) - inc_grp.shift(tau_a)
    # for each group, multiply the difference with the conversion factor
    for i in inc.index.unique('age_group'):
        for j in inc.index.unique('vacc_status'):
            delta[:, i, j] *= alpha[i, j]
    # for each group calculate the cumulative sum
    vij = delta.groupby(level=['age_group', 'vacc_status']).cumsum()
    # calculate total occupation by summing over the groups
    v = vij.groupby(level='t').sum()
    return v
    